- Competition: https://www.kaggle.com/competitions/aerial-cactus-identification/overview
- Dataset: https://www.kaggle.com/competitions/aerial-cactus-identification/data
- Code: https://www.kaggle.com/code/shahules/getting-started-with-cnn-and-vgg16
- Deep Learning: https://www.kaggle.com/learn/intro-to-deep-learning

# Introduction

1. How to prepare your data to feed to a neural network
2. How convolution neural network works
3. How to implement cnn in keras
4. How to evaluate your results
5. How to use keras pretrained network to improve your model
6. How to predit and create a submission

## Loading required limbraries

In [10]:
import pandas as pd
import os, cv2
from IPython.display import Image
from keras.preprocessing import image
from keras import optimizers
from keras import layers, models
from keras.applications.imagenet_utils import preprocess_input
import matplotlib.pyplot as plt
import seaborn as sns
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
print(os.listdir('/Users/kenny_jung/aiffel/data/aerial_cactus/'))

import numpy as np

['.DS_Store', 'test', 'train', 'train.csv', 'sample_submission.csv']


## Setting dataset directories

In [12]:
train_dir = '/Users/kenny_jung/aiffel/data/aerial_cactus/train'
test_dir = '/Users/kenny_jung/aiffel/data/aerial_cactus/test'
train = pd.read_csv('/Users/kenny_jung/aiffel/data/aerial_cactus/train.csv')
df_test = pd.read_csv('/Users/kenny_jung/aiffel/data/aerial_cactus/sample_submission.csv')

In [13]:
df_test.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,0.5
1,0017242f54ececa4512b4d7937d1e21e.jpg,0.5
2,001ee6d8564003107853118ab87df407.jpg,0.5
3,002e175c3c1e060769475f52182583d0.jpg,0.5
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,0.5


## Getting a basic idea

In [14]:
train.head(5)
train.has_cactus = train.has_cactus.astype(str)

In [15]:
print('out dataset has {} rows and {} columns'.format(train.shape[0], train.shape[1]))

out dataset has 17500 rows and 2 columns


In [16]:
train['has_cactus'].value_counts()

has_cactus
1    13136
0     4364
Name: count, dtype: int64

In [18]:
print('The number  of rows in test set is %d'%(len(os.listdir('../Users/kenny_jung/aiffel/data/aerial_cactus/test'))))

FileNotFoundError: [Errno 2] No such file or directory: '../Users/kenny_jung/aiffel/data/aerial_cactus/test'